In [ ]:
#!pip install -U sentence-transformers rank_bm25

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch
import pandas as pd


In [8]:
df = pd.read_json('product_title.json' , lines=True)

In [9]:
df.head()

,text,id
0,Propet Logan Men's Sneakers Black 8 MED,5d510c0a59443bd0c302e850500c1feddc91b13b
1,Propet Logan Men's Sneakers Brown 10.5 MED,a1223430fcd213ceb15c9014d9ba303705b2ee5b
2,Propet Logan Men's Sneakers Black 11.5 MED,640f34f6703cc72c054630bb0edafdb627a58a49
3,Propet Logan Men's Sneakers Black 13 MED,8e56269884f46fdff72e6993eeebc424f00e5c8b
4,Propet Logan Men's Sneakers Black 14 MED,76e8ab3515ff502de2fc84e42c97ae8d4fd21bf3


In [14]:


if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")


#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/794 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [15]:
df.head() 

,text,id
0,Propet Logan Men's Sneakers Black 8 MED,5d510c0a59443bd0c302e850500c1feddc91b13b
1,Propet Logan Men's Sneakers Brown 10.5 MED,a1223430fcd213ceb15c9014d9ba303705b2ee5b
2,Propet Logan Men's Sneakers Black 11.5 MED,640f34f6703cc72c054630bb0edafdb627a58a49
3,Propet Logan Men's Sneakers Black 13 MED,8e56269884f46fdff72e6993eeebc424f00e5c8b
4,Propet Logan Men's Sneakers Black 14 MED,76e8ab3515ff502de2fc84e42c97ae8d4fd21bf3


In [17]:
df.shape

(1943818, 2)

In [20]:
# We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
#corpus_embeddings = bi_encoder.encode(df['text'], convert_to_tensor=True, show_progress_bar=True)

In [21]:
# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

In [22]:
# tokenized_corpus = []
# for passage in tqdm(data['text']):
#     tokenized_corpus.append(bm25_tokenizer(passage))


In [24]:
corpus_embeddings = torch.load("embeddings.pt",map_location=torch.device('cpu') )

In [65]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):
    print("Input question:", query)

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    #question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    #print(hits)
    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, df['id'][hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]


    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-10 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    final_df = []
    
    for hit in hits[0:10]:
        
        id_var = df['id'][hit['corpus_id']].replace("\n", " ")
        test_var = df['text'][hit['corpus_id']].replace("\n", " ")
        score = hit['cross-score']
        d={'id_var':id_var,'text':test_var,'Score':score}
        df_final_new =pd.DataFrame(d,index=[0])
        final_df.append(df_final_new)
        print("\t{}\t{}\t{:.3f}".format(df['id'][hit['corpus_id']].replace("\n", " "),df['text'][hit['corpus_id']].replace("\n", " "),hit['cross-score']))

    final_df=pd.concat(final_df)
    
    return final_df
    

In [66]:
a=search("What brands  of jeans are there for men ")

Input question: What brands  of jeans are there for men 

-------------------------

Top-10 Cross-Encoder Re-ranker hits
	9d0473e6f00bd586a8521453c1588ff261de32fa	Men's Levi's® 513™ Slim Straight Stretch Jeans Emgee 32X29	-10.526
	4b79c71924c61f0f7431710b1cdc39b61640f82e	Men's Levi's® 559™ Relaxed Straight Fit Jeans Range 40X32	-10.677
	2a4a1c4e1093cba12d2181379a65663b2f233017	Men's Levi's® 559™ Relaxed Straight Fit Jeans Black 38X32	-10.687
	262158455d13511685f28215bbbb76b718913c4e	Men's Levi's® 559™ Relaxed Straight Fit Jeans Black 30X32	-10.748
	78ebf400879b5889ec73705e241fa4d6d9d6eabd	Men's Levi's® 559™ Relaxed Straight Fit Jeans Light Blue 38X32	-10.789
	50fa33c7599331178e5de97ef96c1b6276a016d8	Men's Levi's® 559™ Relaxed Straight Fit Jeans Range 30X32	-10.805
	a10e18e908801a186617c1e3f82c0bb9f26c305b	Men's Levi's® 559™ Relaxed Straight Fit Jeans Range 42X32	-10.859
	b7afb19eb1a59ad033a524e54f7a4259b48748ff	Men's Levi's® 514™ Straight Jeans Black Soft Twill 38X30	-10.870
	15a69f780

In [67]:
a

,id_var,text,Score
0,9d0473e6f00bd586a8521453c1588ff261de32fa,Men's Levi's® 513™ Slim Straight Stretch Jeans...,-10.525821
0,4b79c71924c61f0f7431710b1cdc39b61640f82e,Men's Levi's® 559™ Relaxed Straight Fit Jeans ...,-10.676777
0,2a4a1c4e1093cba12d2181379a65663b2f233017,Men's Levi's® 559™ Relaxed Straight Fit Jeans ...,-10.686694
0,262158455d13511685f28215bbbb76b718913c4e,Men's Levi's® 559™ Relaxed Straight Fit Jeans ...,-10.748376
0,78ebf400879b5889ec73705e241fa4d6d9d6eabd,Men's Levi's® 559™ Relaxed Straight Fit Jeans ...,-10.788898
0,50fa33c7599331178e5de97ef96c1b6276a016d8,Men's Levi's® 559™ Relaxed Straight Fit Jeans ...,-10.804819
0,a10e18e908801a186617c1e3f82c0bb9f26c305b,Men's Levi's® 559™ Relaxed Straight Fit Jeans ...,-10.858742
0,b7afb19eb1a59ad033a524e54f7a4259b48748ff,Men's Levi's® 514™ Straight Jeans Black Soft T...,-10.870343
0,15a69f780f2bf1550bcef838174668111e908f91,Men's Levi's® 559™ Relaxed Straight Fit Jeans ...,-10.934950
0,68c7179af6e169eb372616e78a192feb069d3d39,Men's Levi's® 559™ Relaxed Straight Fit Jeans ...,-10.941975


In [68]:
import json 

In [70]:
result = a.to_json(orient="split")
 

In [72]:
parsed = json.loads(result)


In [75]:
parsed=json.dumps(parsed, indent=4,) 

'{\n    "columns": [\n        "id_var",\n        "text",\n        "Score"\n    ],\n    "index": [\n        0,\n        0,\n        0,\n        0,\n        0,\n        0,\n        0,\n        0,\n        0,\n        0\n    ],\n    "data": [\n        [\n            "9d0473e6f00bd586a8521453c1588ff261de32fa",\n            "Men\'s Levi\'s\\u00ae 513\\u2122 Slim Straight Stretch Jeans Emgee 32X29",\n            -10.5258207321\n        ],\n        [\n            "4b79c71924c61f0f7431710b1cdc39b61640f82e",\n            "Men\'s Levi\'s\\u00ae 559\\u2122 Relaxed Straight Fit Jeans Range 40X32",\n            -10.676776886\n        ],\n        [\n            "2a4a1c4e1093cba12d2181379a65663b2f233017",\n            "Men\'s Levi\'s\\u00ae 559\\u2122 Relaxed Straight Fit Jeans Black 38X32",\n            -10.6866941452\n        ],\n        [\n            "262158455d13511685f28215bbbb76b718913c4e",\n            "Men\'s Levi\'s\\u00ae 559\\u2122 Relaxed Straight Fit Jeans Black 30X32",\n            -10